In [ ]:
import gzip
import numpy as np

In [ ]:
def load_mnist_images(filename):
    with gzip.open(filename, 'rb') as f:
        data = np.frombuffer(f.read(), np.uint8, offset=16)
        data = data.reshape(-1, 28, 28)
    return data

def load_mnist_labels(filename):
    with gzip.open(filename, 'rb') as f:
        labels = np.frombuffer(f.read(), np.uint8, offset=8)
    return labels

In [ ]:
# Wczytanie danych treningowych i testowych
train_images = load_mnist_images('train-images-idx3-ubyte.gz')
train_labels = load_mnist_labels('train-labels-idx1-ubyte.gz')
test_images = load_mnist_images('t10k-images-idx3-ubyte.gz')
test_labels = load_mnist_labels('t10k-labels-idx1-ubyte.gz')

In [ ]:
# Normalizacja danych obrazu, aby były między 0 a 1
train_images = train_images / 255.0
test_images = test_images / 255.0

# Dodanie dodatkowego wymiaru (kanału koloru) do danych obrazu
train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)

In [ ]:
# Definiowanie liczby klas i konwersja etykiet na kodowanie one-hot
num_classes = 10
train_labels = np.eye(num_classes)[train_labels]
test_labels = np.eye(num_classes)[test_labels]

In [ ]:
# Wyświetlenie kształtu danych obrazu i etykiet
print(f'train_images shape: {train_images.shape}')
print(f'test_images shape: {test_images.shape}')
print(f'train_labels shape: {train_labels.shape}')
print(f'test_labels shape: {test_labels.shape}')

In [ ]:
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from tensorflow.keras.models import Sequential

def build_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    return model

# Parametry modelu
input_shape = (28, 28, 1)  # Dla obrazów Kazushji-MNIST (KMNIST)
num_classes = 10  # Liczba kategorii w Kazushji-MNIST (różnych znaków)

# Budowanie modelu
model = build_model(input_shape, num_classes)

# Kompilacja modelu
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Podsumowanie i wyświetlenie tabeli z informacjami o modelu
model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

checkpoint_callback = ModelCheckpoint('model.keras', save_best_only=True)
reduce_lr_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

hist = model.fit(train_images, train_labels, epochs=10, batch_size=64, validation_split=0.2, 
                 callbacks=[checkpoint_callback, reduce_lr_callback])

In [ ]:
import matplotlib.pyplot as plt

# Wykres dokładności modelu
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Dokładność modelu')
plt.ylabel('Dokładność')
plt.xlabel('Epoka')
plt.legend(['Trening', 'Walidacja'], loc='upper left')
plt.show()

# Wykres funkcji straty modelu
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Funkcja straty modelu')
plt.ylabel('Wartość funkcji straty')
plt.xlabel('Epoka')
plt.legend(['Trening', 'Walidacja'], loc='upper left')
plt.show()

In [ ]:
# Przewidywanie etykiet dla zbioru testowego
predictions = model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)
print(predictions.shape)
print(predicted_labels)

In [ ]:
def plot_images(images, true_labels, predicted_labels, class_names, num_images=25):
    true_label_indices = np.argmax(true_labels, axis=1)  # Convert from one-hot to class indices
    plt.figure(figsize=(15, 18))
    for i in range(num_images):
        plt.subplot(5, 5, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[i], cmap=plt.cm.binary)
        color = 'green' if predicted_labels[i] == true_label_indices[i] else 'red'
        plt.xlabel(f"True: {class_names[true_label_indices[i]]}\nPred: {class_names[predicted_labels[i]]}", color=color)
    plt.show()

# Nazwy klas (etykiety)
class_names = ['O', 'KI', 'SU', 'TSU', 'NA',
               'HA', 'MA', 'YA', 'RE', 'WO']

# Wywołanie funkcji plot_images
plot_images(test_images[:25], test_labels[:25], predicted_labels[:25], class_names)